# Import

In [32]:
%matplotlib inline
import seaborn as sns
import matplotlib as mpl  # 기본 설정
import matplotlib.pyplot as plt  # 그래프 그리기
import matplotlib.font_manager as fm  # 폰트 관리
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import LambdaCallback, Callback
from sklearn.metrics import r2_score

!apt-get update -qq         # apt-get 패키지 설치 명령어, -qq : 에러외 메세지 숨기기
!apt-get install fonts-nanum* -qq #나눔글꼴 설치

fe = fm.FontEntry(fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumGothic') #파일 저장되어있는 경로와 이름 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) #폰트설정
mpl.rcParams['axes.unicode_minus'] = False

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


# Data Preprocessing

In [33]:
df = pd.read_csv('/content/월별_관광지인구.csv')
df['기준연월'] = pd.to_datetime(df['년도'].astype(str) + df['월'].astype(str).str.zfill(2), format='%Y%m')
# 관광지명별로 스케일링 적용
scalers = {}
scaled_data = []

for name, group in df.groupby('관광지명'):
    group = group.copy()  # 그룹 데이터를 복사하여 처리
    scaler = MinMaxScaler()  # 각 관광지별 스케일러 생성
    group['입장객수_scaled'] = scaler.fit_transform(group[['입장객수']])  # 스케일링
    scalers[name] = scaler  # 스케일러 저장
    scaled_data.append(group)

# 스케일링된 데이터프레임 병합
df_scaled = pd.concat(scaled_data).reset_index(drop=True)

In [34]:
# 관광지별 데이터 준비
X_list, y_list = [], []
sequence_length = 12
for location in df_scaled['관광지명'].unique():  # 관광지별로 데이터 분리
    # 특정 관광지 데이터만 선택
    location_data = df_scaled[df_scaled['관광지명'] == location].sort_values('기준연월')
    location_values = location_data['입장객수_scaled'].values

    # 시계열 데이터 생성
    for i in range(len(location_values) - sequence_length):
        X_list.append(location_values[i:i+sequence_length])  # 12개월 입력
        y_list.append(location_values[i+sequence_length])   # 다음 달 예측값

# 최종 입력 및 출력 배열
X = np.array(X_list).reshape(-1, sequence_length, 1)  # LSTM 입력 차원 (samples, timesteps, features)
y = np.array(y_list)



# Modeling

In [35]:
# 모델 구성
model = Sequential([
    LSTM(256, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)),
    LSTM(128, activation='relu'),
    Dense(1, activation='relu') # 출력 레이어 무조건 0 이상이 되도록 출력 #relu
])

model.compile(optimizer='adam', loss='mse')
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_15 (LSTM)                       │ (None, 12, 256)             │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_16 (LSTM)                       │ (None, 128)                 │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 461,441 (1.76 MB)

 Trainable params: 461,441 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# 관광지별 Train/Validation/Test 분할
X_train, X_val, X_test = [], [], []
y_train, y_val, y_test = [], [], []

sequence_length = 12

for location in df_scaled['관광지명'].unique():
    # 특정 관광지 데이터 선택
    location_data = df_scaled[df_scaled['관광지명'] == location].sort_values('기준연월')
    location_values = location_data['입장객수_scaled'].values

    # 시계열 데이터 생성
    X_list, y_list = [], []
    for i in range(len(location_values) - sequence_length):
        X_list.append(location_values[i:i + sequence_length])  # 12개월 입력
        y_list.append(location_values[i + sequence_length])   # 다음 달 예측값

    # 시계열 데이터를 numpy 배열로 변환
    X = np.array(X_list).reshape(-1, sequence_length, 1)
    y = np.array(y_list)

    # 데이터 크기 계산
    total_size = len(X)
    train_size = int(total_size * 0.7)
    val_size = int(total_size * 0.2)

    # Train, Validation, Test Split
    X_train.extend(X[:train_size])
    y_train.extend(y[:train_size])
    X_val.extend(X[train_size:train_size + val_size])
    y_val.extend(y[train_size:train_size + val_size])
    X_test.extend(X[train_size + val_size:])
    y_test.extend(y[train_size + val_size:])

# 리스트를 numpy 배열로 변환
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)

# 데이터 크기 확인
print(f"Train size: {len(X_train)}")
print(f"Validation size: {len(X_val)}")
print(f"Test size: {len(X_test)}")


Train size: 304
Validation size: 76
Test size: 76


# Train

In [37]:
# 사용자 정의 콜백 클래스
class R2ScoreCallback(Callback):
    def __init__(self, X, y):
        super().__init__()
        self.X = X
        self.y = y

    def on_epoch_end(self, epoch, logs=None):
        # 예측값 계산

        y_pred = self.model.predict(self.X, verbose=0)
        # R-squared 계산
        r2 = r2_score(self.y, y_pred)
        # Loss와 R-squared 출력
        if epoch ==0:
            print(f"Epoch {epoch + 1}, Loss: {logs['loss']:.4f}, Validation Loss: {logs['val_loss']:.4f}, R-squared: {r2:.4f}")

        elif (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch + 1}, Loss: {logs['loss']:.4f}, Validation Loss: {logs['val_loss']:.4f}, R-squared: {r2:.4f}")


# R2ScoreCallback 생성
r2_callback = R2ScoreCallback(X_train, y_train)

# 모델 학습
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=0,
    callbacks=[r2_callback]
)


Epoch 1, Loss: 0.1457, Validation Loss: 0.0533, R-squared: -0.2709
Epoch 20, Loss: 0.0525, Validation Loss: 0.0399, R-squared: 0.2998
Epoch 40, Loss: 0.0397, Validation Loss: 0.0693, R-squared: 0.4194
Epoch 60, Loss: 0.0413, Validation Loss: 0.0456, R-squared: 0.5024
Epoch 80, Loss: 0.0354, Validation Loss: 0.0436, R-squared: 0.5295
Epoch 100, Loss: 0.0334, Validation Loss: 0.0582, R-squared: 0.5294


# Test

In [47]:
# 관광지별 마지막 12개월 데이터로 예측
predictions_dict = {}

for location in df_scaled['관광지명'].unique():
    # 특정 관광지 데이터 선택
    location_data = df_scaled[df_scaled['관광지명'] == location].sort_values('기준연월')
    location_values = location_data['입장객수_scaled'].values  # 스케일된 데이터 사용

    # 마지막 12개월 데이터 준비
    last_sequence = location_values[-sequence_length:]

    # 12개월 예측
    predictions = []
    for _ in range(12):
        next_value = model.predict(last_sequence.reshape(1, sequence_length, 1), verbose=0)
        predictions.append(next_value[0, 0])
        last_sequence = np.append(last_sequence[1:], next_value)

    # 스케일 복원
    predictions = scalers[location].inverse_transform(np.array(predictions).reshape(-1, 1))  # 관광지별 스케일러 사용
    predictions = np.round(predictions)  # 소수점 첫째 자리에서 반올림
    predictions_dict[location] = predictions.flatten()

# 결과 출력
for location, preds in predictions_dict.items():
    print(f"관광지: {location}, 내년 예측값: {preds}")

#유명한거 / 떠오르는거

관광지: 거북선전시관, 내년 예측값: [4201. 4048. 4653. 5016. 5023. 4810. 3969. 4127. 3797. 4033. 3055. 2860.]
관광지: 남해 양떼목장 양모리학교, 내년 예측값: [4245. 4330. 4435. 4811. 5372. 5071. 4638. 4558. 5027. 5158. 5056. 4830.]
관광지: 남해국제탈공연예술촌, 내년 예측값: [ 662.  610.  573.  664.  763.  827.  978. 1119. 1085.  927. 1098.  983.]
관광지: 남해군 요트학교, 내년 예측값: [119. 133. 137. 142. 197. 216. 199. 161. 146. 145. 151. 155.]
관광지: 남해스포츠파크, 내년 예측값: [ 8033.  9860.  9087.  9392. 11065. 10785. 10967. 10844.  8911.  8527.
  8264.  8085.]
관광지: 남해유배문학관, 내년 예측값: [5122. 5307. 5727. 6225. 6443. 7511. 6353. 6684. 6214. 6797. 5933. 6324.]
관광지: 남해편백자연휴양림, 내년 예측값: [15101. 15436. 15948. 17132. 16996. 17837. 17703. 15550. 17300. 18192.
 15853. 14762.]
관광지: 노도, 내년 예측값: [624. 650. 707. 766. 692. 708. 765. 814. 884. 916. 788. 709.]
관광지: 독일마을, 내년 예측값: [52404. 55675. 62620. 74678. 76914. 73508. 68100. 76375. 70167. 89620.
 64804. 60033.]
관광지: 사우스케이프(골프장), 내년 예측값: [2954. 3427. 3577. 4171. 4536. 4630. 3931. 4148. 4349. 4778. 4546. 3440.]
관광지: 섬이정원, 내년 예측값:

In [48]:
dates = pd.date_range(start="2024-01-01", periods=12, freq="MS")

# 데이터프레임 생성
data = []
for location, predictions in predictions_dict.items():
    for month, value in zip(dates, predictions):
        data.append({'관광지명': location, '기준연월': month, '예측한 입장객수': value})

result_df = pd.DataFrame(data)

In [49]:
result_data = []

for name, group in result_df.groupby('관광지명'):
    group = group.copy()  # 그룹 데이터를 복사하여 처리
    scaler = MinMaxScaler()  # 각 관광지별 스케일러 생성
    group['예측한 입장객수_scaled'] = scaler.fit_transform(group[['예측한 입장객수']])  # 스케일링
    scalers[name] = scaler  # 스케일러 저장
    result_data.append(group)

# 스케일링된 데이터프레임 병합|
result_df_scaled = pd.concat(result_data).reset_index(drop=True)

# 입장객수를 기준으로 사용자에 따라 관광지 추천
- 스케일된 기준: 꼭 가야하는 관광지
- 스케일 안된 기준: 유명한 관광지

In [58]:
top_3_result = result_df.groupby('기준연월').apply(lambda x: x.nlargest(3, '예측한 입장객수')).reset_index(drop=True)
scaled_top_3_result = result_df_scaled.groupby('기준연월').apply(lambda x: x.nlargest(3, '예측한 입장객수_scaled')).reset_index(drop=True)
comparison = pd.concat(
    [
        scaled_top_3_result[['기준연월', '관광지명', '예측한 입장객수']].rename(columns={'예측한 입장객수': '스케일된 입장객수'}),
        top_3_result[['관광지명', '예측한 입장객수']].rename(columns={'입장객수': '예측한 입장객수'})
    ],
    axis=1
)

comparison

<ipython-input-58-ba438eb9b03a>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_3_result = result_df.groupby('기준연월').apply(lambda x: x.nlargest(3, '예측한 입장객수')).reset_index(drop=True)
<ipython-input-58-ba438eb9b03a>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  scaled_top_3_result = result_df_scaled.groupby('기준연월').apply(lambda x: x.nlargest(3, '예측한 입장객수_scaled')).reset_index(drop=True)


,기준연월,관광지명,스케일된 입장객수,관광지명,예측한 입장객수
0,2024-01-01,거북선전시관,4201.0,한려해상국립공원 금산(복곡주차장),61902.0
1,2024-01-01,조도,1380.0,독일마을,52404.0
2,2024-01-01,한려해상국립공원 금산(복곡주차장),61902.0,파독전시관,32905.0
3,2024-02-01,남해스포츠파크,9860.0,한려해상국립공원 금산(복곡주차장),64740.0
4,2024-02-01,한려해상국립공원 금산(복곡주차장),64740.0,독일마을,55675.0
5,2024-02-01,거북선전시관,4048.0,파독전시관,35047.0
6,2024-03-01,거북선전시관,4653.0,한려해상국립공원 금산(복곡주차장),67296.0
7,2024-03-01,조도,1433.0,독일마을,62620.0
8,2024-03-01,한려해상국립공원 금산(복곡주차장),67296.0,파독전시관,36170.0
9,2024-04-01,한려해상국립공원 금산(복곡주차장),70180.0,독일마을,74678.0
